# Last FM hometask <br>
https://www.kaggle.com/ravichaubey1506/lastfm <br>
1. Выбрать данные по странам своей группы (совместно): <br>
    3530203_70101: Germany, Netherlands <br>
    3530203_70102: Belarus, Ukraine, Poland, Russian Federation<br>
    3530903_70301: Sweden, Finland, Norway, Denmark, Iceland<br>
    3530903_70302: Spain, Portugal, France, Italy, Belgium<br>
    
2. Попытаться найти полезные с точки зрения продвижения групп (или еще чего-нибудь) и нетривиальные правила, используя алгоритмы Apriori, FPGrowth, FPMax и всевозможные метрики. Хотя бы 5 правил.
3. Вывести эти правила в отдельных ячейках. 
4. Подумать, как можно было бы использовать полученные правила.

In [1]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

In [2]:
data = pd.read_csv("lastfm.csv")
data.head()

,user,artist,sex,country
0,1,red hot chili peppers,f,Germany
1,1,the black dahlia murder,f,Germany
2,1,goldfrapp,f,Germany
3,1,dropkick murphys,f,Germany
4,1,le tigre,f,Germany


In [3]:
countries = 'Sweden, Finland, Norway, Denmark, Iceland'.split(', ')
countries

['Sweden', 'Finland', 'Norway', 'Denmark', 'Iceland']

In [4]:
lastfm_df = data[data['country'].isin(countries)]
len(lastfm_df)

30268

In [58]:
user_info = lastfm_df[['user', 'country', 'sex']].drop_duplicates('user')
user_info.set_index('user', inplace=True)
user_info.head()

,country,sex
user,,
5,Finland,m
7,Finland,m
19,Sweden,f
28,Sweden,m
48,Norway,m


In [6]:
# dummy encoding исполнителя
pivoted_artist = lastfm_df[['user', 'artist']].pivot_table(index='user', columns='artist',aggfunc=any, fill_value=False).astype(int)
pivoted_artist.sum() # количество непустых значений

artist
...and you will know us by the trail of dead    17
2pac                                            49
3 doors down                                    58
30 seconds to mars                              57
311                                              3
                                                ..
yeah yeah yeahs                                 14
yellowcard                                      15
yo la tengo                                      8
zero 7                                          23
Édith piaf                                      14
Length: 995, dtype: int64

In [7]:
# dummy encoding страны
pivoted_country = lastfm_df[['user', 'country']].pivot_table(index='user', columns='country',aggfunc=any, fill_value=False).astype(int)
pivoted_country.sum()

country
Denmark    119
Finland    510
Iceland     16
Norway     214
Sweden     575
dtype: int64

In [146]:
# dummy encoding пола пользователя
pivoted_sex = lastfm_df[['user', 'sex']].pivot_table(index='user', columns='sex',aggfunc=any, fill_value=False).astype(int)
pivoted_sex.sum()

sex
f     332
m    1102
dtype: int64

In [61]:
pivoted_all = pd.concat([pivoted_artist, pivoted_country, pivoted_sex], axis=1)
pivoted_all.head()

,...and you will know us by the trail of dead,2pac,3 doors down,30 seconds to mars,311,36 crazyfists,44,50 cent,65daysofstatic,[unknown],...,yo la tengo,zero 7,Édith piaf,Denmark,Finland,Iceland,Norway,Sweden,f,m
user,,,,,,,,,,,,,,,,,,,,,
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
19,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,1,0
28,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
48,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1


In [68]:
pivoted_all[pivoted_all.columns[-13:]]

,yann tiersen,yeah yeah yeahs,yellowcard,yo la tengo,zero 7,Édith piaf,Denmark,Finland,Iceland,Norway,Sweden,f,m
user,,,,,,,,,,,,,
5,0,0,0,0,0,0,0,1,0,0,0,0,1
7,0,0,0,0,0,0,0,1,0,0,0,0,1
19,0,0,0,1,0,0,0,0,0,0,1,1,0
28,0,0,0,0,0,0,0,0,0,0,1,0,1
48,0,0,0,0,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19582,0,0,0,0,0,0,0,0,0,1,0,0,1
19586,0,0,0,0,0,0,0,0,0,0,1,0,1
19587,0,0,0,0,0,0,1,0,0,0,0,0,1


In [69]:
frequent_itemsets = apriori(pivoted_all, min_support=0.05, use_colnames=True)

frequent_itemsets

,support,itemsets
0,0.087866,(ac/dc)
1,0.069735,(air)
2,0.070432,(arctic monkeys)
3,0.054393,(belle and sebastian)
4,0.064854,(björk)
...,...,...
169,0.050907,"(m, Sweden, in flames)"
170,0.050907,"(m, Sweden, kent)"
171,0.068340,"(Finland, m, metallica)"
172,0.063459,"(Finland, m, nightwish)"


In [70]:
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.5)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(ac/dc),(m),0.087866,0.768480,0.076709,0.873016,1.136030,0.009185,1.823222
1,(air),(m),0.069735,0.768480,0.052999,0.760000,0.988966,-0.000591,0.964668
2,(bob dylan),(Sweden),0.089958,0.400976,0.052999,0.589147,1.469282,0.016928,1.458001
3,(bob dylan),(m),0.089958,0.768480,0.071827,0.798450,1.038999,0.002696,1.148696
4,(bruce springsteen),(m),0.059275,0.768480,0.051604,0.870588,1.132871,0.006052,1.789020
...,...,...,...,...,...,...,...,...,...
67,"(Sweden, kent)",(m),0.078801,0.768480,0.050907,0.646018,0.840644,-0.009650,0.654045
68,"(Finland, metallica)",(m),0.076709,0.768480,0.068340,0.890909,1.159314,0.009391,2.122269
69,"(Finland, nightwish)",(m),0.076709,0.768480,0.063459,0.827273,1.076506,0.004510,1.340380
70,"(m, nightwish)",(Finland),0.112971,0.355649,0.063459,0.561728,1.579448,0.023281,1.470210


In [138]:
# правила, единственным следствием которых является страна
country_rules = rules[rules['consequents'].map(lambda x: x.issubset(countries))]
country_rules_index = list(country_rules.index)
country_rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
2,(bob dylan),(Sweden),0.089958,0.400976,0.052999,0.589147,1.469282,0.016928,1.458001
6,(coldplay),(Sweden),0.172245,0.400976,0.086471,0.502024,1.252005,0.017405,1.202918
15,(håkan hellström),(Sweden),0.069038,0.400976,0.059972,0.868687,2.166430,0.032290,4.561796
18,(johnny cash),(Sweden),0.097629,0.400976,0.053696,0.550000,1.371652,0.014549,1.331164
21,(kent),(Sweden),0.120642,0.400976,0.078801,0.653179,1.628972,0.030426,1.727185


In [139]:
# правила, единственным следствием которых является пол
sex_rules = rules[rules['consequents'].map(lambda x: x.issubset(['m', 'f']))]
sex_rules_index = list(sex_rules.index)
sex_rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(ac/dc),(m),0.087866,0.76848,0.076709,0.873016,1.136030,0.009185,1.823222
1,(air),(m),0.069735,0.76848,0.052999,0.760000,0.988966,-0.000591,0.964668
3,(bob dylan),(m),0.089958,0.76848,0.071827,0.798450,1.038999,0.002696,1.148696
4,(bruce springsteen),(m),0.059275,0.76848,0.051604,0.870588,1.132871,0.006052,1.789020
5,(children of bodom),(m),0.078103,0.76848,0.068340,0.875000,1.138612,0.008320,1.852162


In [147]:
# правила, следствия которых содержат исполнителя
artist_rules = rules[~rules.index.isin(sex_rules_index + country_rules_index)]
artist_rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction


In [142]:
# правила с наибольшим confidence
rules.sort_values('confidence', ascending=False).head(5)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
39,(rage against the machine),(m),0.069038,0.76848,0.065551,0.949495,1.235550,0.012497,4.584100
17,(iron maiden),(m),0.092050,0.76848,0.085774,0.931818,1.212547,0.015035,3.395630
53,(the prodigy),(m),0.064854,0.76848,0.059972,0.924731,1.203325,0.010133,3.075912
65,"(Sweden, in flames)",(m),0.056485,0.76848,0.050907,0.901235,1.172750,0.007499,2.344142
45,(soilwork),(m),0.060669,0.76848,0.054393,0.896552,1.166656,0.007770,2.238029


In [152]:
# правила с наибольшим support
rules.sort_values('support', ascending=False).head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
60,(Sweden),(m),0.400976,0.76848,0.306137,0.763478,0.993492,-0.002005,0.978854
58,(Finland),(m),0.355649,0.76848,0.273361,0.768627,1.000192,0.000053,1.000638
26,(metallica),(m),0.164575,0.76848,0.147141,0.894068,1.163424,0.020669,2.185551
16,(in flames),(m),0.154812,0.76848,0.132497,0.855856,1.113700,0.013527,1.606172
59,(Norway),(m),0.149233,0.76848,0.117852,0.789720,1.027639,0.003170,1.101007


In [154]:
# правила, включающие название артиста, с наибольшим support
artist_rules.sort_values('support', ascending=False).head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
